In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
"""
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric
"""

'\n!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html\n!pip install torch-geometric\n'

In [4]:
from torch.autograd import Variable
import torch_geometric.transforms
from torch_geometric.transforms import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import torch
import pyarrow as pa
import pyarrow.parquet as pq
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from torch_geometric.nn import knn_graph
import os 

from torch.utils.data import *
from functools import partial
import timeit

In [5]:
import argparse
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--maxnodes', type=int, default=185, help='Max Nodes')
parser.add_argument('--epochs', type=int, default=50, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=50, help='Initial learning rate.') #100

parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.') #0.001
parser.add_argument('--dropout', type=float, default=0.3, help='Dropout rate (1 - keep probability).')
args = parser.parse_args([])
torch.backends.cudnn.benchmark = True

In [6]:
from torch.utils.data import *
class ParquetDataset(Dataset):
    def __init__(self, filename):
        self.parquet = pq.ParquetFile(filename)
        self.cols = None 
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        # Preprocessing
        #data['X'] = np.float32(data['X'][0]) 
        data['X'] = torch.Tensor(data['X'][0])
        return dict(data)
    def __len__(self):
        return self.parquet.num_row_groups

In [7]:
def get_data_loader(datasets, batch_size, cut, random_sampler=False):
    dset = ConcatDataset([ParquetDataset(dataset) for dataset in datasets])
    idxs = np.random.permutation(len(dset))
    if random_sampler: 
        random_sampler = sampler.SubsetRandomSampler(idxs[:cut])
    else: 
        random_sampler = None 
    data_loader = DataLoader(dataset=dset, batch_size=batch_size, shuffle=False, num_workers=10, sampler=random_sampler, pin_memory=True)
    return data_loader

In [8]:
cd drive/My\ Drive

/content/drive/My Drive


In [9]:

datasets = ['ElectronTrainData.h5.snappy.parquet','PhotonTrainData.h5.snappy.parquet']
data_loader = get_data_loader(datasets, args.batch_size, cut = None, random_sampler = True)

In [10]:

val_datasets = ['PhotonValData.h5.snappy.parquet','ElectronValData.h5.snappy.parquet']
val_loader = get_data_loader(val_datasets, args.batch_size, cut = None, random_sampler = True)

In [11]:

def jets(datei,labels,number1,number2):

    cols = None

    graphs=[]

    for i in range(number1,number2):

      ecal=datei[i].cuda()

      xhit2,yhit2=torch.nonzero(ecal,as_tuple=True)  ## Select hits in detector

      eneEcal=ecal[xhit2,yhit2]*50 ## Select energies of hits

      feats=torch.stack((xhit2.float(),yhit2.float(),eneEcal),dim=1) ## concatenate x,y locations and energies (3 features in total)

      cords=feats[:,[0,1]] ## cords = x,y coordinates 

      #if cords.shape[0]>=10: ## just to make sure a sample is not empty 

      #feats[:,0]/=125
      #feats[:,1]/=125

      edge_index = knn_graph(cords, k=6, batch=None, loop=True)  ## Create knn graph adjacency matrix 

      donnees=Data(x=feats,edge_index=edge_index,label=labels[i]) ## Create graph data with feature matrix x and adjacency matrix edge_index
      
      graphs.append(donnees)

    return graphs

In [12]:
import ep_model

In [13]:
model=ep_model.GraphClass(3,64,128,128,256,128,64,args.maxnodes)
model.train()
model.to(device)#.cuda()

GraphClass(
  (sage1): DenseSAGEConv(3, 64)
  (sage2): DenseSAGEConv(64, 128)
  (poolit1): Linear(in_features=64, out_features=50, bias=True)
  (poolit2): Linear(in_features=128, out_features=15, bias=True)
  (sage3): DenseSAGEConv(64, 128)
  (sage5): DenseSAGEConv(128, 128)
  (tr1): Linear(in_features=128, out_features=64, bias=True)
  (tr2): Linear(in_features=64, out_features=1, bias=True)
  (fin): Linear(in_features=15, out_features=1, bias=True)
  (drop4): Dropout(p=0.4, inplace=False)
  (drop3): Dropout(p=0.3, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (bano1): BatchNorm1d(185, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano2): BatchNorm1d(185, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano3): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bano5): BatchNorm1d(15, eps=1e-05, momentum=0.1, affi

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

In [15]:

## generate list to count nodes for each graph
def nodeCounter(samples):
    inds=[]
    for k in samples:
        inds.append(k['x'].shape[0])
    return inds

In [16]:
def ref(bsize,nodeC,i1,i2):
  maxC=np.max(np.array(nodeC))
  maxC=args.maxnodes#maxC + (4 - maxC % 4) ##max num of nodes 1161%4
  refMat=np.zeros((bsize,maxC)) ## matrix of zeros
  for pi in range(i1,i2):##10
    refMat[bsize-(i2-pi),:nodeC[pi]]=1 ## fill ones 
  return refMat,maxC

In [17]:
def assigner(nodelist):
  fin=[]
  countit=0
  for m in nodelist:
      fin.append(np.repeat(countit,m))
      countit+=1
  return np.array(fin)

In [18]:
from optimizer import loss_function

In [19]:
def get_accuracy(y_true, y_prob):
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    y_prob = y_prob > 0.5
    return (y_true == y_prob).sum().item() / y_true.size(0)

In [ ]:
import time
from torch_geometric.utils import to_dense_batch, to_dense_adj
for epoch in range(50):
      #model.train()
  count=0
  c1,c2=0,args.batch_size
  epLoss=0
  t = time.time()
  for i, data in enumerate(data_loader):
        ecal2 = data['X'][:,0,:,:].cuda()
        labels=data['y'][0].cuda()
        rawGraph=jets(ecal2,labels,0,args.batch_size) ##Generating graphs from raw data 
        nodeCount=nodeCounter(rawGraph)
        lengs=torch.LongTensor(np.hstack(assigner(np.array(nodeCount[c1:c2])))).cuda()
        
        compress=torch_geometric.data.Batch.from_data_list(rawGraph)

        gra=compress.x.to(device).clone()
        adj=compress.edge_index.to(device).clone()

        count+=1
        #refMat,maxCount=ref(args.batch_size,nodeCount,c1,c2)

        whole,mask=to_dense_batch(gra, lengs, fill_value=0, max_num_nodes=args.maxnodes)#refMat.shape[1])
        wholeAdj=to_dense_adj(adj, lengs, edge_attr=None, max_num_nodes=args.maxnodes)#refMat.shape[1]).cuda()
        
        whole=whole.cuda()
        wholeAdj=wholeAdj.cuda()
        mask=mask.cuda()
        lengs=lengs.cuda()

        optimizer.zero_grad()

        r1= model(whole,wholeAdj,lengs,mask,args.maxnodes)

        loss = loss_function(r1,labels.unsqueeze_(1))/args.batch_size    

        loss.backward()

        optimizer.step()

        cur_loss = loss.item()
        
        epLoss+=float(cur_loss)

        #c1+=args.batch_size
        #c2+=args.batch_size
        if count%3200==0:
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(epLoss/count),"time=", "{:.5f}".format(time.time() - t))
            t = time.time()
            print(get_accuracy(labels.squeeze_(1),r1.squeeze_(1)))

  torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch':epoch,
        'loss': loss,
        'epLoss':epLoss
        }, './loadBatches_b16_pool3.pth')


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
gra